<a href="https://colab.research.google.com/github/dominiquebuford/AIPI540-project1/blob/dombuford/detectron2_naiveFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install necessary dependencies
#install detectron2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Some basic setup:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import cv2
import json
from tqdm.notebook import tqdm

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import ColorMode
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# For reading annotations file
from pycocotools.coco import COCO

# utilities
from pprint import pprint
from collections import OrderedDict
import os

# For data visualisation
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from google.colab.patches import cv2_imshow

In [ ]:
def register_dataset(datasetName, annotationsPath, imagesPath):
  register_coco_instances(datasetName, {}, annotationsPath, imagesPath)

def set_up_configuration(trainedModelToUse):
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file(trainedModelToUse))
  #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(trainedModelToUse)
  return cfg

def makePredictions(cfg, datasetName):
  predictor = DefaultPredictor(cfg)
  evaluator = COCOEvaluator(datasetName, cfg, False, output_dir = "./output")

  val_loader = build_detection_test_loader(cfg, datasetName)
  valResults = inference_on_dataset(predictor.model, val_loader, evaluator)


In [ ]:
"COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

VAL_ANNOTATIONS_PATH= '/content/Project_1/annotations.json'  #replace with the path for validation annotation json file
VAL_IMAGES_PATH = '/content/Project_1/S3_Data/Validation'    #replace with path to Validation image

register_dataset('validation_dataset', VAL_ANNOTATIONS_PATH, VAL_IMAGES_PATH)
cfg = set_up_configuration("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")


makePredictions(cfg, "validation_dataset")



In [ ]:
#evaluate with the naive model on the on the validation set

val_metadata = MetadataCatalog.get("validation_dataset")

im = cv2.imread("/content/Project_1/S3_Data/Validation/008135.jpg").  #replace with path for a singular validation image to see the prediction detections

outputs = predictor(im)

v = Visualizer(im[:, :, ::-1],
                   metadata=val_metadata,
                   scale=2,
                   instance_mode=ColorMode.IMAGE_BW
    )

out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])